# Weather Data

In [2]:
import math
import time
import random
import urllib
import _pickle as pickle  # using cPickle
import os.path
import datetime
import ipywidgets
import ipyparallel

import pandas as pd
import seaborn as sns
import numpy as np

from IPython.display import clear_output

## Weather Data Documentation

**Reference the docs here:**
* https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/readme.txt
* https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/ghcn-daily-by_year-format.rtf
* https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt

The second is a .rtf (Rich Text Format file; think Word) and it's contexts are copied below if you cannot open it:

The following information serves as a definition of each field in one line of data covering one station-day. Each field described below is separated by a comma ( , ) and follows the order
presented in this document.

* ID = 11 character station identification code
* YEAR/MONTH/DAY = 8 character date in YYYYMMDD format (e.g. 19860529 = May 29, 1986)
* ELEMENT = 4 character indicator of element type 
* DATA VALUE = 5 character data value for ELEMENT 
* M-FLAG = 1 character Measurement Flag 
* Q-FLAG = 1 character Quality Flag 
* S-FLAG = 1 character Source Flag 
* OBS-TIME = 4-character time of observation in hour-minute format (i.e. 0700 =7:00 am)

See section III of the GHCN-Daily readme.txt file for an explanation of ELEMENT codes and their units as well as the M-FLAG, Q-FLAGS and S-FLAGS.

The OBS-TIME field is populated with the observation times contained in NOAA/NCDC’s Multinetwork Metadata System (MMS).

In [11]:
with open('pickles/stations_df.pkl', 'rb') as file:
    stations_df = pickle.load(file)

In [12]:
stations_df = stations_df[stations_df['ID'].str.startswith('US') & (stations_df['HCN/CRN FLAG'] == 'HCN')].reset_index(drop=True)

In [13]:
stations_df.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
0,USC00011084,31.0583,-87.0550,25.9,AL,BREWTON 3 SSE,NaN,HCN,NaN
1,USC00012813,30.5467,-87.8808,7.0,AL,FAIRHOPE 2 NE,NaN,HCN,NaN
2,USC00013160,32.8347,-88.1342,38.1,AL,GAINESVILLE LOCK,NaN,HCN,NaN
3,USC00013511,32.7019,-87.5814,67.4,AL,GREENSBORO,NaN,HCN,NaN
4,USC00013816,31.8814,-86.2503,132.0,AL,HIGHLAND HOME,NaN,HCN,NaN


In [14]:
def date_format(yearmonthday):
    return pd.Timestamp(datetime.datetime.strptime(str(yearmonthday), '%Y%m%d'))

In [15]:
for year in range(1980, 2010):
    print(f'downloading weather data for: {year}')
    clear_output(wait=True)
    if os.path.isfile(f'weather_data/{year}.csv.gz'):
        continue
    url = f'http://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/{year}.csv.gz'
    urllib.request.urlretrieve(url, f'weather_data/{year}.csv.gz')

downloading weather data for: 2009


In [16]:
# dict from year to cleaned weather dataframe
weather_df_dict = dict()

In [17]:
%%time
# TODO: parallelize the below (maybe)
# no need to repeat after pickling (only ever do once)
# time required estimate: 20-30+ minutes
years = range(1980, 2010)
for year in years:
    print(f'cleaning dataframe for: {year} ({year-1980+1}/{len(years)})')
    clear_output(wait=True)
    df = pd.read_csv(
        f'weather_data/{year}.csv.gz',
        header=None,
        names=['ID', 'YEARMONTHDAY', 'ELEMENT', 'DATA VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
    )
    df = df[df['ID'].isin(stations_df['ID']) & (df['ELEMENT'].isin(['TMAX', 'PRCP']))]
    df = df.assign(date=np.vectorize(date_format)(df['YEARMONTHDAY']))
    df.drop(columns=['YEARMONTHDAY', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME'], inplace=True)
    weather_df_dict[year] = df

CPU times: user 13min 42s, sys: 33.8 s, total: 14min 16s
Wall time: 12min 36s


In [18]:
with open('pickles/weather_df_dict.pkl', 'wb') as file:
    pickle.dump(weather_df_dict, file)

In [19]:
with open('pickles/weather_df_dict.pkl', 'rb') as file:
    weather_df_dict = pickle.load(file)

In [20]:
temp_dfs = list()
for year in range(1980, 2010):
    temp_df = weather_df_dict[year]
    temp_dfs.append(temp_df[temp_df['ELEMENT'] == 'TMAX'].drop(columns=['ELEMENT']))

In [21]:
# df with all stations' tempurature time series
all_temp_df = pd.concat(temp_dfs)
all_temp_df.reset_index(drop=True, inplace=True)
all_temp_df = all_temp_df.pivot(index='date', columns='ID', values='DATA VALUE')
all_temp_df = all_temp_df / 10 * 9.0 / 5.0 + 32 # convert tenths of celsius to fahrenheit

In [22]:
all_temp_df.head()

ID,USC00011084,USC00012813,USC00013160,USC00013511,USC00013816,USC00017157,USC00017304,USC00017366,USC00018024,USC00018178,...,USW00093805,USW00093806,USW00093808,USW00093986,USW00094008,USW00094224,USW00094728,USW00094793,USW00094794,USW00094967
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,53.96,48.02,46.04,53.06,46.94,44.96,46.04,53.06,57.02,44.96,...,48.92,53.06,37.94,55.04,35.96,53.06,44.96,42.98,33.08,26.96
1980-01-02,62.06,55.04,53.06,59.00,48.92,48.02,44.06,60.08,57.02,57.02,...,60.98,59.00,37.94,51.98,23.00,51.08,42.08,39.02,32.00,19.94
1980-01-03,66.92,NaN,57.92,62.96,60.08,57.02,NaN,62.06,51.98,62.96,...,66.02,60.08,35.06,35.06,35.96,42.98,37.94,37.94,23.00,17.06
1980-01-04,60.98,66.92,59.00,55.94,64.94,50.00,60.08,57.02,55.94,60.98,...,59.00,51.98,33.98,33.98,24.08,42.98,30.02,33.08,23.00,21.92
1980-01-05,50.00,44.96,37.94,44.96,44.96,37.94,48.02,44.96,51.98,42.08,...,50.00,42.98,33.08,35.06,19.04,46.04,32.00,33.08,30.02,23.00


In [23]:
# no need to repeat above steps after the following pickle
with open('pickles/all_temp_df.pkl', 'wb') as file:
    pickle.dump(all_temp_df, file)

## Step 2: Estimating bias-corrected temperature time series at each station

### Part A: Calculate station intercepts to minimize absolute differences between stations

In [24]:
with open('pickles/fips_stations_dict.pkl', 'rb') as file:
    fips_stations_dict = pickle.load(file)

In [25]:
with open('pickles/all_temp_df.pkl', 'rb') as file:
    all_temp_df = pickle.load(file)

In [38]:
all_temp_df.head()

ID,USC00011084,USC00012813,USC00013160,USC00013511,USC00013816,USC00017157,USC00017304,USC00017366,USC00018024,USC00018178,...,USW00093805,USW00093806,USW00093808,USW00093986,USW00094008,USW00094224,USW00094728,USW00094793,USW00094794,USW00094967
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,53.96,48.02,46.04,53.06,46.94,44.96,46.04,53.06,57.02,44.96,...,48.92,53.06,37.94,55.04,35.96,53.06,44.96,42.98,33.08,26.96
1980-01-02,62.06,55.04,53.06,59.00,48.92,48.02,44.06,60.08,57.02,57.02,...,60.98,59.00,37.94,51.98,23.00,51.08,42.08,39.02,32.00,19.94
1980-01-03,66.92,NaN,57.92,62.96,60.08,57.02,NaN,62.06,51.98,62.96,...,66.02,60.08,35.06,35.06,35.96,42.98,37.94,37.94,23.00,17.06
1980-01-04,60.98,66.92,59.00,55.94,64.94,50.00,60.08,57.02,55.94,60.98,...,59.00,51.98,33.98,33.98,24.08,42.98,30.02,33.08,23.00,21.92
1980-01-05,50.00,44.96,37.94,44.96,44.96,37.94,48.02,44.96,51.98,42.08,...,50.00,42.98,33.08,35.06,19.04,46.04,32.00,33.08,30.02,23.00


In [27]:
def fips_stations_intercepts(fips, stations):
    num_stations = len(stations)
    # no adjustments can be made for fips with 1 or fewer stations
    if num_stations == 0:
        return (fips, None)
    elif num_stations == 1:
        return (fips, [0])
    # df with time series for only the stations in the fips
    fips_stations_df = all_temp_df[stations]
    # cache both reported indeces to avoid repeated intersection computation
    both_reported_cache = dict()
    # array of current intercepts; updated for remaining stations every iteration
    curr = np.full(num_stations, 0, dtype=float)
    # array of intercepts before most recent update; used to determine convergence
    last = np.full(num_stations, np.inf, dtype=float)
    it = 1
    # element-wise 'is close' to determine convergence
    while not np.all(np.isclose(last, curr)):
        time.sleep(0)
#         print(f'iterations: {it}')
#         clear_output(wait=True)
        # reference station
        ref_station = random.randrange(0, num_stations)
        # remaining stations
        rem_stations = [station for station in range(0, num_stations) if station != ref_station]
        for rem_station in rem_stations:
            time.sleep(0)
            try:
                # series for indexing the df on days where both stations reported
                both_reported = both_reported_cache[tuple(sorted([ref_station, rem_station]))]
            except KeyError:
                # intersection computation
                both_reported = fips_stations_df[stations[ref_station]].notnull() & fips_stations_df[stations[rem_station]].notnull()
                both_reported_cache[tuple(sorted([ref_station, rem_station]))] = both_reported
            # sum of boolean series is the simpy the number of Trues
            n = np.sum(both_reported)
            # only keep rows (days) where both stations reported
            both_reported_df = fips_stations_df[both_reported]
            # extract series for reference station
            ref_station_reported = both_reported_df[stations[ref_station]]
            # extract series for remaining station
            rem_station_reported = both_reported_df[stations[rem_station]]
            # add previous round's intercepts to respective series
            ref_station_adjusted = ref_station_reported + curr[ref_station]
            rem_station_adjusted = rem_station_reported + curr[rem_station]
            diff = ref_station_adjusted - rem_station_adjusted
            # record previous round's intercept before updating
            last[rem_station] = curr[rem_station]
            # update intercept
            curr[rem_station] = curr[rem_station] + (1 / n) * np.sum(diff)
        it += 1
        if it > 10_000:
            return (fips, 'max iterations exceeded')
    # curr is a list of intercepts
    return (fips, curr)

In [28]:
# indeces correspond
fips_codes = list(fips_stations_dict.keys())
stations = list(fips_stations_dict.values())

In [39]:
rc = ipyparallel.Client()
view = rc.load_balanced_view()

with rc[:].sync_imports():
    import time
    import random

# https://stackoverflow.com/questions/37414206/ipyparallel-view-sync-import-does-not-bind-to-additional-name
%px import numpy as np

rc[:].push(dict(
    all_temp_df=all_temp_df
))

importing time on engine(s)
importing random on engine(s)


/home/avi/anaconda3/lib/python3.7/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@desk')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


<AsyncResult: _push>

In [40]:
ar = view.map_async(fips_stations_intercepts, fips_codes, stations)

In [41]:
num_fips_codes = len(fips_codes)

In [42]:
# ctrl + enter on this cell to see progress without blocking kernel
print(f'Time Elapsed: {datetime.timedelta(seconds=math.ceil(ar.elapsed))}')
print(f'FIPS Codes Completed: {ar.progress}/{num_fips_codes}')
print(f'Completed {round(ar.serial_time/ar.wall_time, 2)}x faster than serial computation' if ar.ready() else 'Still Running')

Time Elapsed: 0:00:07
FIPS Codes Completed: 113/3015
Still Running


In [47]:
# will block the kernel; interrupt the kernel to unblock
ar.wait_interactive()
print(f'Completed {round(ar.serial_time/ar.wall_time, 2)}x faster than serial computation' if ar.ready() else 'Still Running')

3015/3015 tasks finished after  825 s
done
Completed 11.47x faster than serial computation


In [48]:
fips_stations_intercepts_dict = dict()
# the intercepts for these fips codes are not converging
fips_codes_to_check = list()
for fips, intercepts in ar:
    # TODO: what to do with fips with no stations?
    if intercepts is None:
        continue
    elif intercepts == 'max iterations exceeded':
        fips_codes_to_check.append(fips)
        continue
    stations = fips_stations_dict[fips]
    for station, intercept in zip(stations, intercepts):
        try:
            fips_stations_intercepts_dict[fips][station] = intercept
        except KeyError:
            fips_stations_intercepts_dict[fips] = {station: intercept}

/home/avi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [49]:
# TODO: analyze manually
len(fips_codes_to_check)

20

In [50]:
with open('pickles/fips_stations_intercepts_dict.pkl', 'wb') as file:
    pickle.dump(fips_stations_intercepts_dict, file)

### Part B: Adjust all intercepts to reflect typical county weather conditions

In [51]:
with open('pickles/fips_stations_inverse_distances_dict.pkl', 'rb') as file:
    fips_stations_inverse_distances_dict = pickle.load(file)

In [52]:
with open('pickles/fips_stations_intercepts_dict.pkl', 'rb') as file:
    fips_stations_intercepts_dict = pickle.load(file)

In [53]:
for fips, stations_intercepts_dict in fips_stations_intercepts_dict.items():
    stations_inverse_distances_dict = fips_stations_inverse_distances_dict[fips]
    weights = np.array(list(stations_inverse_distances_dict.values()))
    intercepts = np.array(list(stations_intercepts_dict.values()))
    weighted_average = np.sum(weights * intercepts) / np.sum(weights)
    for station in stations_intercepts_dict.keys():
        fips_stations_intercepts_dict[fips][station] -= weighted_average

## Step 3: Calculating weighted average weather for each county

In [54]:
fips_temp_dict = dict()
num_fips_codes = len(fips_stations_dict)
for i, (fips, stations) in enumerate(fips_stations_dict.items()):
    print(f'FIPS Codes: {i+1}/{num_fips_codes}')
    clear_output(wait=True)
    try:
        stations_intercepts_dict = fips_stations_intercepts_dict[fips]
    # occurs only when fips has no stations
    except KeyError:
        continue
    stations_inverse_distances_dict = fips_stations_inverse_distances_dict[fips]
    fips_stations_df = all_temp_df[stations]
    intercepts = np.array(list(stations_intercepts_dict.values()))
    if isinstance(intercepts[0], str):
        continue
    weights = np.array(list(stations_inverse_distances_dict.values()))
    # adjust each station's temp series by the calculated intercept
    fips_stations_df += intercepts
    # apply calculated weights to each station's temp series
    fips_stations_df *= weights
    # take weighted average of each station
    fips_temp_dict[fips] = fips_stations_df.sum(axis=1) / np.sum(weights)
fips_temp_df = pd.DataFrame(fips_temp_dict)

FIPS Codes: 3015/3015


In [55]:
fips_temp_df.head()

,27077,53073,53047,53019,16021,30101,30105,30091,38023,38013,...,12099,12071,48427,12051,48489,48215,12021,12011,12086,12087
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,18.712920,36.685883,32.000951,25.918505,35.831735,45.079819,36.754023,30.515072,25.709490,24.98,...,67.618750,64.837862,67.282964,66.043967,64.04,66.407741,72.301503,59.468211,23.376757,56.614433
1980-01-02,11.071802,38.492549,31.004428,24.825643,37.158181,43.951076,24.790933,19.102348,16.391507,15.98,...,64.426049,63.727822,71.060000,64.250718,71.06,71.060000,68.241944,55.292479,21.693056,51.098822
1980-01-03,7.251181,35.231312,27.912066,24.301328,37.655729,37.492174,35.184564,28.116247,21.955243,21.02,...,63.711470,71.047895,77.000000,67.186110,77.00,77.000000,67.490662,60.555283,24.399004,55.907304
1980-01-04,16.446460,32.616167,24.994008,22.791679,32.033837,34.412249,24.238771,25.893898,22.443736,21.92,...,73.427716,69.110040,61.175555,69.793249,62.06,61.414252,72.059558,65.103943,25.721912,59.018674
1980-01-05,22.254658,30.571894,26.852318,23.645513,31.599524,28.359593,15.929085,16.641174,15.285754,15.08,...,68.551042,65.280600,65.135555,66.904356,66.02,65.374252,68.732165,56.903173,22.053849,53.503063


In [56]:
with open('pickles/fips_temp_df.pkl', 'wb') as file:
    pickle.dump(fips_temp_df, file)

In [57]:
# TODO: repeat for precipitation